取得 Account ID 驗證 AWS 本機設置正確完成

In [1]:
import boto3

# 載入 .env 檔案中的環境變數
from dotenv import load_dotenv
import os
load_dotenv()

# 建立 STS 客戶端
sts_client = boto3.client("sts")

# 呼叫 get_caller_identity 以取得當前 AWS 帳號資訊
response = sts_client.get_caller_identity()
account_id = response["Account"]

print(f"AWS Account ID: {account_id}")

AWS Account ID: 682683043554


遍歷所有 S3 bucket 中的資源並顯示樹狀結構

In [2]:
import boto3

# 建立 S3 資源對象
s3 = boto3.resource('s3')

def list_s3_bucket_structure(bucket_name):
    bucket = s3.Bucket(bucket_name)
    print(f"Bucket: {bucket_name}")
    
    has_objects = False
    for obj in bucket.objects.all():
        print(" └── ", obj.key)
        has_objects = True

    if not has_objects:
        print(" └── <此 bucket 為空>")

# 取得所有 bucket 名稱並顯示每個 bucket 的樹狀結構
s3_client = boto3.client("s3")
buckets = s3_client.list_buckets()["Buckets"]

if buckets:
    for bucket in buckets:
        list_s3_bucket_structure(bucket["Name"])
else:
    print("當前為空的：沒有任何 S3 bucket")

Bucket: c133864a3391508l8212786t1w68268304355-lambdabucket-mlvbnwvfnjyd
 └──  s3cleaner_function.zip
Bucket: c133864a3391508l8212786t1w682683043554-labbucket-zmynq4cbdjiw
 └── <此 bucket 為空>


查詢 Comprehend 權限的代碼

In [7]:
import boto3
from botocore.exceptions import ClientError

# 建立 IAM 和 STS 客戶端
iam_client = boto3.client("iam")
sts_client = boto3.client("sts")

# 取得當前角色名稱
try:
    identity = sts_client.get_caller_identity()
    arn = identity["Arn"]
    # 從 ARN 中解析角色名稱
    role_name = arn.split("/")[-1]
    print(f"當前角色名稱: {role_name}")
except ClientError as e:
    print("無法取得當前角色資訊。錯誤訊息:", e)
    role_name = None

# 列出當前角色的附加策略並檢查 Comprehend 權限
if role_name:
    try:
        policies = iam_client.list_attached_role_policies(RoleName=role_name)
        comprehend_permissions = ["comprehend:DetectKeyPhrases", "comprehend:DetectEntities"]
        has_comprehend_access = False

        for policy in policies["AttachedPolicies"]:
            # 取得每個策略的詳細內容
            policy_arn = policy["PolicyArn"]
            policy_version = iam_client.get_policy_version(
                PolicyArn=policy_arn,
                VersionId=iam_client.get_policy(PolicyArn=policy_arn)["Policy"]["DefaultVersionId"]
            )

            # 檢查策略中的權限
            for statement in policy_version["PolicyVersion"]["Document"]["Statement"]:
                actions = statement.get("Action", [])
                # 若該策略授權 comprehend 權限
                if any(action in comprehend_permissions for action in actions):
                    has_comprehend_access = True
                    print(f"權限來源策略: {policy['PolicyName']}")
                    break

        if has_comprehend_access:
            print("當前角色具備 Comprehend 相關權限。")
        else:
            print("當前角色不具備 Comprehend 相關權限。")
    except ClientError as e:
        print("無法取得當前角色的策略資訊。錯誤訊息:", e)


當前角色名稱: user2069478=_________
無法取得當前角色的策略資訊。錯誤訊息: An error occurred (NoSuchEntity) when calling the ListAttachedRolePolicies operation: The role with name user2069478=_________ cannot be found.


嘗試看看

In [11]:
import boto3
import json
import re

# 設定 AWS Comprehend 客戶端
comprehend_client = boto3.client("comprehend")

# 讀取標準化後的文本內容
normalized_file_path = "transcribe_output_std.txt"
with open(normalized_file_path, "r") as file:
    text = file.read()

# 確認文本長度，Comprehend 對單次請求文本的長度限制是 5000 字符
max_length = 5000
if len(text) > max_length:
    text = text[:max_length]  # 截取前 5000 字符

# 初始化結果變數
key_phrases = []
entities = []

# 1. 提取關鍵詞組
try:
    key_phrases_response = comprehend_client.detect_key_phrases(
        Text=text,
        LanguageCode="en"
    )
    key_phrases = key_phrases_response["KeyPhrases"]
    print("關鍵詞組提取結果:")
    for phrase in key_phrases:
        print(f"- {phrase['Text']} (信心分數: {phrase['Score']:.2f})")
except Exception as e:
    print("提取關鍵詞組時出錯:", e)

# 2. 主題檢測
try:
    entities_response = comprehend_client.detect_entities(
        Text=text,
        LanguageCode="en"
    )
    entities = entities_response["Entities"]
    print("\n主題檢測結果:")
    for entity in entities:
        print(f"- {entity['Text']} (類型: {entity['Type']}, 信心分數: {entity['Score']:.2f})")
except Exception as e:
    print("主題檢測時出錯:", e)

# 3. 將結果保存到新文件中
output_file_path = "comprehend_analysis_result.json"
analysis_results = {
    "KeyPhrases": key_phrases if key_phrases else "無法取得關鍵詞組",
    "Entities": entities if entities else "無法取得主題"
}
with open(output_file_path, "w") as output_file:
    json.dump(analysis_results, output_file, ensure_ascii=False, indent=4)

print("\n分析結果已保存至:", output_file_path)


提取關鍵詞組時出錯: An error occurred (AccessDeniedException) when calling the DetectKeyPhrases operation: User: arn:aws:sts::682683043554:assumed-role/voclabs/user2069478=_________ is not authorized to perform: comprehend:DetectKeyPhrases because no identity-based policy allows the comprehend:DetectKeyPhrases action
主題檢測時出錯: An error occurred (AccessDeniedException) when calling the DetectEntities operation: User: arn:aws:sts::682683043554:assumed-role/voclabs/user2069478=_________ is not authorized to perform: comprehend:DetectEntities because no identity-based policy allows the comprehend:DetectEntities action

分析結果已保存至: comprehend_analysis_result.json
